# How to Get YouTube Video Transcripts with yt-dlp

In this guide, we'll show you how to get YouTube transcripts.

For this, we use the [yt-dlp](https://github.com/yt-dlp/yt-dlp) library to download YouTube videos and then transcribe it with the AssemblyAI API.

`yt-dlp` is a [youtube-dl](https://github.com/ytdl-org/youtube-dl) fork with additional features and fixes. It is better maintained and preferred over `youtube-dl` nowadays.

In this guide we'll show 3 different approaches:

- Option 1: Download video via CLI
- Option 2: Download video via code
- Option 3: Extract audio url without downloading

Let's get started!

## Install Dependencies

Install [yt-dlp](https://github.com/yt-dlp/yt-dlp) and the [AssemblyAI Python SDK](https://github.com/AssemblyAI/assemblyai-python-sdk) via pip.

In [ ]:
!pip install -U yt-dlp

In [ ]:
!pip install assemblyai

## Option 1: Download video via CLI

In this approach we download the YouTube video via the command line and then transcribe it via the AssemblyAI API. We use the following video here:

- https://www.youtube.com/watch?v=wtolixa9XTg

To download it, use the `yt-dlp` command with the following options:

- `-f m4a/bestaudio`: The format should be the best audio version in m4a format.
- `-o "%(id)s.%(ext)s"`: The output name should be the id followed by the extension. In this example, the video gets saved to "wtolixa9XTg.m4a".
- `wtolixa9XTg`: the id of the video.

In [4]:
!yt-dlp -f m4a/bestaudio -o "%(id)s.%(ext)s" wtolixa9XTg

[youtube] Extracting URL: wtolixa9XTg
[youtube] wtolixa9XTg: Downloading webpage
[youtube] wtolixa9XTg: Downloading ios player API JSON
[youtube] wtolixa9XTg: Downloading android player API JSON
[youtube] wtolixa9XTg: Downloading m3u8 information
[info] wtolixa9XTg: Downloading 1 format(s): 140
[download] Destination: wtolixa9XTg.m4a
[download] 100% of    7.14MiB in 00:00:00 at 16.32MiB/s
[FixupM4a] Correcting container of "wtolixa9XTg.m4a"


Next, set up the AssemblyAI SDK and trancribe the file. Replace `YOUR_API_KEY` with your own key. If you don't have one, you can [sign up here](https://assemblyai.com/dashboard/signup) for free.

Make sure that the path you pass to the `transcribe()` function corresponds to the saved filename.

In [5]:
import assemblyai as aai

aai.settings.api_key = "YOUR_API_KEY"

transcriber = aai.Transcriber()
transcript = transcriber.transcribe("wtolixa9XTg.m4a")

In [14]:
transcript.text

"Hi everyone, I'm Patrick and in this video I show you how I would learn machine learning if I could start over. For context, I'm a machine learning developer advocate at Assemblyai and before that I worked several years as software developer and ML Engineer, and I also teach Python and machine learning on my own YouTube channel. So I would say I'm pretty experienced in the field, but I know that the available courses out there can be overwhelming for beginners, so I hope to give you some guide ends with this video. The demand for machine learning engineers is still increasing every year, so it's a great skill to have. I divided this learning path into seven steps that should take you about three months to finish. Of course, this can vary depending on how much time and effort you want to put into this, and I know that everyone learns differently or might have different goals. So this is just my personal take on how to learn machine learning. You can use this guide if you just want to e

## Option 2: Download video via code

In this approach we download the video with a Python script instead of the command line.

You can download the file with the following code:

In [7]:
import yt_dlp

URLS = ['https://www.youtube.com/watch?v=wtolixa9XTg']

ydl_opts = {
    'format': 'm4a/bestaudio/best',  # The best audio version in m4a format
    'outtmpl': '%(id)s.%(ext)s',  # The output name should be the id followed by the extension
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }]
}


with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(URLS)

[youtube] Extracting URL: https://www.youtube.com/watch?v=wtolixa9XTg
[youtube] wtolixa9XTg: Downloading webpage
[youtube] wtolixa9XTg: Downloading ios player API JSON
[youtube] wtolixa9XTg: Downloading android player API JSON
[youtube] wtolixa9XTg: Downloading m3u8 information
[info] wtolixa9XTg: Downloading 1 format(s): 140
[download] wtolixa9XTg.m4a has already been downloaded
[download] 100% of    7.13MiB
[ExtractAudio] Not converting audio wtolixa9XTg.m4a; file is already in target format m4a


After downloading, you can again use the code from option 1 to transcribe the file:

In [ ]:
import assemblyai as aai

aai.settings.api_key = "YOUR_API_KEY"

transcriber = aai.Transcriber()
transcript = transcriber.transcribe("wtolixa9XTg.m4a")

## Option 3: Extract audio url without downloading

In this approach, we don't download the file at all. Instead, we use `yt-dlp` to extract a publicly accessible url of the video that the AssemblyAI API can handle.

First, extract the video info with this code:

In [8]:
import yt_dlp

URL = 'https://www.youtube.com/watch?v=wtolixa9XTg'

with yt_dlp.YoutubeDL() as ydl:
    info = ydl.extract_info(URL, download=False)

[youtube] Extracting URL: https://www.youtube.com/watch?v=wtolixa9XTg
[youtube] wtolixa9XTg: Downloading webpage
[youtube] wtolixa9XTg: Downloading ios player API JSON
[youtube] wtolixa9XTg: Downloading android player API JSON
[youtube] wtolixa9XTg: Downloading m3u8 information


Next, iterate over all formats. The formats are already sorted worst to best, so we iterate in reverse and stop at the first m4a file. This is the format with the best audio version.

The url to the hosted file can then be accessed with the `url` key.

In [10]:
# formats are already sorted worst to best
# --> iterate in reverse and take the first audio to get the best
for format in info["formats"][::-1]:
  if format["resolution"] == "audio only" and format["ext"] == "m4a":
    url = format["url"]
    break

print(url)

https://rr3---sn-q4fzenee.googlevideo.com/videoplayback?expire=1700490576&ei=8BhbZfvACJWzsfIPqZ6U8Ag&ip=34.125.111.1&id=o-AMLjZ2ZLO-E2194RF4kTVtjFbzFbkD9P5GDPvHQ2bSt3&itag=140&source=youtube&requiressl=yes&mh=AE&mm=31%2C26&mn=sn-q4fzenee%2Csn-a5mekn6s&ms=au%2Conr&mv=u&mvi=3&pl=21&vprv=1&svpuc=1&mime=audio%2Fmp4&gir=yes&clen=7486557&dur=462.541&lmt=1689983079290840&mt=1700467761&fvip=2&keepalive=yes&fexp=24007246&beids=24350018&c=IOS&txp=4432434&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cvprv%2Csvpuc%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&sig=ANLwegAwRQIhANdpXltSWFLSbLmBx3DS5Uv5oHlmtRmxTlfdykktYMC-AiA9-y9RTaB4cZdtezzaNMzzQaVTTsBdLM2T_7WoZzNP7Q%3D%3D&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl&lsig=AM8Gb2swRQIgLDvNSnFWE_g6XnHwnq8CuoMfz59pLCvdCvOX2K1C45ECIQC98tb6CrE21vfHdYhJ4Zr7W0rpFax2qrtNdT12CEhD2Q%3D%3D


Now, use the same AssemblyAI SDK code again to transcribe the file. The `transcribe()` function can handle both a path to a local file, or a publicly accessible url like here. Hence, the code is the same as in the other approaches:

In [12]:
import assemblyai as aai

aai.settings.api_key = "YOUR_API_KEY"

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(url)

In [13]:
transcript.text

"Hi everyone, I'm Patrick and in this video I show you how I would learn machine learning if I could start over. For context, I'm a machine learning developer advocate at Assemblyai and before that I worked several years as software developer and ML Engineer, and I also teach Python and machine learning on my own YouTube channel. So I would say I'm pretty experienced in the field, but I know that the available courses out there can be overwhelming for beginners, so I hope to give you some guide ends with this video. The demand for machine learning engineers is still increasing every year, so it's a great skill to have. I divided this learning path into seven steps that should take you about three months to finish. Of course, this can vary depending on how much time and effort you want to put into this, and I know that everyone learns differently or might have different goals. So this is just my personal take on how to learn machine learning. You can use this guide if you just want to e